In [5]:
import pandas as pd
import time

from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist as pdist

In [3]:
df = pd.read_csv('/global/homes/b/bpb/Downloads/20160414_MTX_avg_greater10.csv')

In [4]:
leaf_names = df.orf.tolist()

In [9]:
df.head()

,orf,timepoint=11.5h-afterwetup_MTX,timepoint=15min-afterwetup_MTX,timepoint=18h-afterwetup_MTX,timepoint=1h-afterwetup_MTX,timepoint=2h-afterdrydown_MTX,timepoint=3d-afterdrydown_MTX,timepoint=3min-afterwetup_MTX,timepoint=72h-priortodrydown_MTX,timepoint=9h-afterwetup_MTX
0,k101_188_2_2,2.8000,23.6545,2.6425,9.322000,1.404667,1.2815,1.458333,1.964667,2.291667
1,k101_26631_689_5,1.8220,41.5085,3.0175,17.063000,4.698667,1.5315,1.037000,6.833333,1.062333
2,k101_38613_971_2,1.8645,6.8680,1.6635,12.746333,2.512000,1.1220,2.385000,1.391667,2.304000
3,k101_39069_982_2,4.0455,20.5485,2.6600,15.799667,1.147667,1.1205,1.874000,1.451333,2.169333
4,k101_44297_1105_1,4.5460,29.3305,4.9625,15.501333,5.520333,3.4760,6.236000,5.963333,5.056667


In [18]:
keys = df.keys()
M = df.as_matrix(columns = keys[1:])

In [19]:
distance_matrix = pdist(M,'correlation')

In [20]:


t0 = time.time()
Z = hierarchy.linkage(distance_matrix,method='average')

print time.time() - t0


4.32454514503


In [21]:
def getNewick(node, newick, parentdist, leaf_names):
    if node.is_leaf():
        return "%s:%.5f%s" % (leaf_names[node.id], parentdist - node.dist, newick)
    else:
        if len(newick) > 0:
            newick = "):%.5f%s" % (parentdist - node.dist, newick)
        else:
            newick = ");"
        newick = getNewick(node.get_left(), newick, node.dist, leaf_names)
        newick = getNewick(node.get_right(), ",%s" % (newick), node.dist, leaf_names)
        newick = "(%s" % (newick)
        return newick

In [22]:
import sys
sys.setrecursionlimit(2000) #whoa!
tree = hierarchy.to_tree(Z,False)
my_str = getNewick(tree, "", tree.dist, leaf_names)
# print my_str
with open('tls_output_tree.txt','w') as fid:
    fid.write('%s'%my_str)